## Attributi sull'intero testo

- Dimensione del vocabolario (V)
- Lunghezza del testo in numero di parole (T)
- Rapporto V/T
- Entropia (H)

In [1]:
import math

def word_counter(RDD):
    '''
    Data una RDD, conta quante volte compare ogni parola ritornando anche la dimensione del vocabolario.
    
    Parameters
    ----------
    RDD : RDD
        RDD del file in input

    Returns
    -------
    (RDD, int)
        RDD dell'output del word count e dimensione del vocabolario
    '''
    
    word_counter = (RDD.flatMap(lambda x: x)
                    .map(lambda x: (x,1))
                    .reduceByKey(lambda a,b: a+b)
                    .sortBy(lambda x: -x[1])
                   )
    
    return word_counter, word_counter.count()

def text_length_in_words(RDD_word_counter):
    '''
    Calcola la lunghezza del testo in termini di numero di parole.
    
    Parameters
    ----------
    RDD_word_counter : RDD
        RDD dell'output del word count

    Returns
    -------
    int
        numero di parole totali presenti nel testo
    '''

    return (RDD_word_counter.map(lambda x: x[1])
            .reduce(lambda a,b: a+b)
           )

def entropy(RDD_word_counter, text_len):
    '''
    Calcola l'entropia (numero medio di bit richiesti per rappresentare tutte le parole del testo).
    
    Parameters
    ----------
    RDD_word_counter : RDD
        RDD dell'output del word count
    text_len : int
        numero di parole totali presenti nel testo

    Returns
    -------
    float
        valore dell'entropia
    '''
    
    return -(RDD_word_counter.map(lambda x: (x[1]/text_len) * math.log2(x[1]/text_len))
             .reduce(lambda a,b: a+b)
            )    # l'entropia ha segno negativo

## Attributi sulle frasi

- Lunghezza massima di una frase (MSL)
- Lunghezza media di una frase (ASL)
- Lunghezza minima di una frase (mSL)
- Distribuzione di probabilità delle lunghezze delle frasi (PDSL)
- Probabilità della lunghezza di frase più frequente (pMFSL)

In [20]:
def sentence_lengths(RDD):
    '''
    Ritorna una lista che contiene le lunghezze (in termini di numero di parole) di tutte le frasi di un testo.
    
    Parameters
    ----------
    RDD : RDD
        RDD del file in input

    Returns
    -------
    list
        lunghezze delle frasi
    '''
    
    # OPERAZIONI PRELIMINARI SUL TESTO
    text = RDD.flatMap(lambda x: x).reduce(lambda a,b: a + ' ' + b) # METTO TUTTO IL TESTO IN UNA STRINGA UNICA
    text = text.replace("?", ".") # ? TERMINA UNA FRASE
    text = text.replace("!", ".") # ! TERMINA UNA FRASE
    text = text.split('. ') # SPLITTO QUANDO TROVO UN CARATTERE CHE TERMINA UNA FRASE (. SEGUITO DA UNO SPAZIO)
    
    return (sc.parallelize(text)
            .map(lambda x: len(x.split(' ')))
            .collect()
           ) # PER OGNI FRASE TROVATA CONTO LE SUE PAROLE

def prob_distr_of_sentence_length(RDD_sen_len):
    '''
    Ritorna una lista che contiene la distribuzione di probabilità delle lunghezze delle frasi.
    
    Parameters
    ----------
    RDD_sen_len : RDD
        RDD dell'output di sentence_lengths

    Returns
    -------
    list
        distribuzione di probabilità delle lunghezze delle frasi
    '''
    
    tot = RDD_sen_len.count()

    return (RDD_sen_len.map(lambda x: (x,1))
            .reduceByKey(lambda a,b: a+b)
            .map(lambda x: (x[0], x[1]/tot))
            .sortBy(lambda x: -x[1])
           )

## Attributi sulla probabilità delle parole

- Distribuzione di probabilità delle 30 parole più comuni (PkMCW)
- Probabilità della parola più comune escludendo 'and' e 'the' (pMCW)
- Probabilità della parola più comune escludendo articoli e preposizioni (pMCWx)
- Probabilità della parola 'the' (pThe)
- Probabilità della virgola (pComma)

In [3]:
def prob_distr_of_30_most_common_words(RDD_word_counter, text_len):
    '''
    Ritorna la distribuzione di probabilità delle 30 parole più comuni.
    
    Parameters
    ----------
    RDD_word_counter : RDD
        RDD dell'output del word count
    text_len : int
        numero di parole totali presenti nel testo

    Returns
    -------
    RDD
        RDD che contiene la distribuzione di probabilità
    '''
    
    return sc.parallelize(RDD_word_counter.map(lambda x: (x[0], x[1]/text_len)).take(30))

def prob_of_the_most_common_word(RDD_word_counter, text_len):
    '''
    Ritorna la probabilità della parola più comune (escludendo 'and' e 'the').
    
    Parameters
    ----------
    RDD_word_counter : RDD
        RDD dell'output del word count
    text_len : int
        numero di parole totali presenti nel testo

    Returns
    -------
    RDD
        RDD che contiene la probabilità della MCW
    '''
    
    return sc.parallelize(prob_distr_of_30_most_common_words(RDD_word_counter, text_len)
                          .filter(lambda x: x[0] != "and" and x[0] != "the")
                          .take(1)
                         )

def prob_of_the_most_common_word_x(RDD_word_counter, text_len):
    '''
    Ritorna la probabilità della parola più comune (escludendo articoli e preposizioni).
    
    Parameters
    ----------
    RDD_word_counter : RDD
        RDD dell'output del word count
    text_len : int
        numero di parole totali presenti nel testo

    Returns
    -------
    RDD
        RDD che contiene la probabilità della MCW
    '''
    
    prep_art = open("preposizioni_e_articoli.txt").read().splitlines()
    
    return sc.parallelize(prob_distr_of_30_most_common_words(RDD_word_counter, text_len)
                          .filter(lambda x: x[0] not in prep_art)
                          .take(1)
                         )

def prob_of_The(RDD_word_counter, text_len):
    '''
    Ritorna la probabilità della parola 'The'.
    
    Parameters
    ----------
    RDD_word_counter : RDD
        RDD dell'output del word count
    text_len : int
        numero di parole totali presenti nel testo

    Returns
    -------
    RDD
        RDD che contiene la probabilità di 'The'
    '''
    
    return (RDD_word_counter.filter(lambda x: x[0] == "the")
            .map(lambda x: (x[0], x[1]/text_len))
           )

def prob_of_comma(RDD_sentences_data, text_len):
    '''
    Ritorna la probabilità di presenza della virgola.
    
    Parameters
    ----------
    RDD_sentences_data : RDD
        RDD del file in input
    text_len : int
        numero di parole totali presenti nel testo

    Returns
    -------
    int
        probabilità di presenza della virgola
    '''
    
    return (RDD_sentences_data.flatMap(lambda x: x)
            .filter(lambda x: "," in x)
            .count()
           ) / text_len

## Attributi sulla distanza

- Distanza media/minima/massima tra apparenze consecutive della MCW (adMCW, mdMCW, MsMCW)
- Distanza media/minima/massima tra apparenze consecutive della MCW escludendo articoli e preposizioni (adMCWx, mdMCWx, MsMCWx)
- Distanza media/minima/massima tra apparenze consecutive di 'the' (adThe, mdThe, MsThe)
- Distanza media/minima/massima tra apparenze consecutive della virgola (adComma, mdComma, MsComma)

In [44]:
def distance_consec_appear(RDD, word):
    '''
    Ritorna una lista che contiene le distanze tra apparenze consecutive di word.
    
    Parameters
    ----------
    RDD : RDD
        RDD del file in input
    word : str
        parola da trattare

    Returns
    -------
    list
        distanze tra apparenze consecutive di word
    '''
    
    if word == ',':
        vect_pos = (RDD.flatMap(lambda x:x)
                    .zipWithIndex()
                    .filter(lambda x: ',' in x[0])
                    .map(lambda x: x[1])
                    .collect()
                   )
    else:
        vect_pos = (RDD.flatMap(lambda x:x)
                    .zipWithIndex()
                    .filter(lambda x: x[0] == word)
                    .map(lambda x: x[1])
                    .collect()
                   )
    
    vect_dis = []
    
    for i in range(1, len(vect_pos)):
        vect_dis.append(vect_pos[i] - vect_pos[i-1])
    
    return vect_dis

# MAIN

In [4]:
def remove_number_some_punctuation_marks(row):
    lowercase = row.lower()
    lowercase = lowercase.replace("--", " ")
    
    res = ""
    
    for char in lowercase:
        if not ('0' <= char <= '9' or char == '"'):
            res += char

    return res

def remove_number_punctuation_marks(row):
    lowercase = row.lower()
    lowercase = lowercase.replace("--", " ")
    
    res = ""
    
    for char in lowercase:
        if 'a' <= char <= 'z' or char == ' ' or char == '-' or char == "'":
            res += char

    return res

def load_file_without_punctuations_marks(filepath):
    # caricamento del dataset
    raw_text = sc.textFile(filepath)

    # rimuoviamo i numeri e i segni di punteggiatura
    
    return (raw_text.filter(bool)                    # rimuoviamo le stringhe vuote
        .map(remove_number_punctuation_marks)
        .map(lambda x : ' '.join(x.split()))        # rimuoviamo diversi spazi bianchi con uno
        .map(lambda row : row.split(" "))
       )

def load_file_without_number(filepath):
    # caricamento del dataset
    raw_text = sc.textFile(filepath)

    # rimuoviamo i numeri e i segni di punteggiatura
    
    return (raw_text.filter(bool)                    # rimuoviamo le stringhe vuote
        .map(remove_number_some_punctuation_marks)
        .map(lambda x : ' '.join(x.split()))        # rimuoviamo diversi spazi bianchi con uno
        .map(lambda row : row.split(" "))
       )

def getCollection(RDD):
    return RDD.collect()

def getValue(RDD):
    return RDD.collect()[0]


data = load_file_without_punctuations_marks("datasets/Anthony Trollope___The O'Conors of Castle Conor from Tales from all Countries.txt")
data.persist()

PythonRDD[2] at RDD at PythonRDD.scala:53

In [7]:
RDD_word_counter, vocabulary_size = word_counter(data)
print("V:\t", vocabulary_size)

text_length = text_length_in_words(RDD_word_counter)
print("T:\t", text_length)

print("V/T:\t", vocabulary_size/text_length)

entropy_value = entropy(RDD_word_counter, text_length)
print("H:\t", entropy_value)

V:	 1558
T:	 7653
V/T:	 0.20358029530902913
H:	 8.605861060321123


In [5]:
sentences_data = load_file_without_number("datasets/Anthony Trollope___The O'Conors of Castle Conor from Tales from all Countries.txt")
sentences_data.persist()

PythonRDD[5] at RDD at PythonRDD.scala:53

In [22]:
sen_lengths = sentence_lengths(sentences_data)
print("MSL:\t", max(sen_lengths))
print("ASL:\t", sum(sen_lengths)/len(sen_lengths))
print("mSL:\t", min(sen_lengths))

RDD_sen_lengths = sc.parallelize(sen_lengths)

prob_distr_freq_sen = prob_distr_of_sentence_length(RDD_sen_lengths)
print("PDSL:\t", getCollection(prob_distr_freq_sen))
print("pMFSL:\t", getValue(prob_distr_freq_sen))

MSL:	 84
ASL:	 14.862135922330097
mSL:	 1
PDSL:	 [(7, 0.06407766990291262), (2, 0.05631067961165048), (9, 0.04660194174757282), (10, 0.04660194174757282), (17, 0.04271844660194175), (12, 0.040776699029126215), (8, 0.038834951456310676), (16, 0.038834951456310676), (1, 0.038834951456310676), (3, 0.038834951456310676), (15, 0.038834951456310676), (6, 0.036893203883495145), (11, 0.03495145631067961), (5, 0.03495145631067961), (13, 0.03300970873786408), (4, 0.031067961165048542), (20, 0.02912621359223301), (25, 0.02330097087378641), (22, 0.02330097087378641), (18, 0.021359223300970873), (14, 0.021359223300970873), (19, 0.019417475728155338), (29, 0.019417475728155338), (23, 0.019417475728155338), (21, 0.017475728155339806), (24, 0.015533980582524271), (26, 0.013592233009708738), (28, 0.013592233009708738), (35, 0.011650485436893204), (30, 0.009708737864077669), (31, 0.009708737864077669), (27, 0.007766990291262136), (41, 0.005825242718446602), (33, 0.005825242718446602), (59, 0.00582524271

In [9]:
RDD_prob_distr_of_30 = prob_distr_of_30_most_common_words(RDD_word_counter, text_length)
print("PkMCW:\t", getCollection(RDD_prob_distr_of_30))

RDD_prob_the_most_common_word_1 = prob_of_the_most_common_word(RDD_word_counter, text_length)
print("pMCW:\t", getValue(RDD_prob_the_most_common_word_1))

RDD_prob_the_most_common_word_2 = prob_of_the_most_common_word_x(RDD_word_counter, text_length)
print("pMCWx:\t", getValue(RDD_prob_the_most_common_word_2))

RDD_prob_the = prob_of_The(RDD_word_counter, text_length)
print("pThe:\t", getValue(RDD_prob_the))

p_comma = prob_of_comma(sentences_data, text_length)
print("pComma:\t", p_comma)

PkMCW:	 [('the', 0.04494969293087678), ('i', 0.036717627074349925), ('and', 0.033320266562132494), ('to', 0.023650855873513656), ('of', 0.021298837057363126), ('a', 0.019730824513262774), ('my', 0.01620279628903698), ('that', 0.016072128577028617), ('in', 0.015941460865020254), ('was', 0.01528812230497844), ('said', 0.012152097216777734), ('as', 0.011106755520710831), ('he', 0.011106755520710831), ('at', 0.009930746112635568), ('but', 0.009669410688618842), ('you', 0.009277407552593753), ('for', 0.009146739840585392), ('me', 0.00862406899255194), ('it', 0.00862406899255194), ('had', 0.007840062720501764), ('with', 0.00718672416045995), ('not', 0.006533385600418136), ("o'conor", 0.006272050176401411), ('all', 0.006141382464393048), ('his', 0.005880047040376323), ('were', 0.00574937932836796), ('on', 0.005618711616359597), ('so', 0.004965373056317784), ('there', 0.004704037632301058), ('we', 0.004704037632301058)]
pMCW:	 ('i', 0.036717627074349925)
pMCWx:	 ('i', 0.036717627074349925)
pTh

In [51]:
MCW = getValue(RDD_prob_the_most_common_word_1)[0]
dist_consec_MCW = distance_consec_appear(data, MCW)
print("adMCW:\t", sum(dist_consec_MCW)/len(dist_consec_MCW))
print("mdMCW:\t", min(dist_consec_MCW))
print("MdMCW:\t", max(dist_consec_MCW))

MCWx = getValue(RDD_prob_the_most_common_word_1)[0]
dist_consec_MCWx = distance_consec_appear(data, MCWx)
print("adMCWx:\t", sum(dist_consec_MCWx)/len(dist_consec_MCWx))
print("mdMCWx:\t", min(dist_consec_MCWx))
print("MdMCWx:\t", max(dist_consec_MCWx))

dist_consec_the = distance_consec_appear(data, 'the')
print("adThe:\t", sum(dist_consec_the)/len(dist_consec_the))
print("mdThe:\t", min(dist_consec_the))
print("MdThe:\t", max(dist_consec_the))

dist_consec_comma = distance_consec_appear(sentences_data, ',')
print("adComma:", sum(dist_consec_comma)/len(dist_consec_comma))
print("mdComma:", min(dist_consec_comma))
print("MdComma:", max(dist_consec_comma))

adMCW:	 27.15357142857143
mdMCW:	 2
MdMCW:	 177
adMCWx:	 27.15357142857143
mdMCWx:	 2
MdMCWx:	 177
adThe:	 22.244897959183675
mdThe:	 2
MdThe:	 115
adComma: 14.713733075435202
mdComma: 1
MdComma: 92
